In [12]:
import random
import pandas as pd
import numpy as np
import os

# Seed for reproducibility
random.seed(42)
np.random.seed(42)

In [13]:
# Define equipment types, manufacturers, and models
equipment_classes = ["HOIST", "UG_CONVEYOR", "MILL", "PUMP", "FAN", "MOTOR"]
manufacturers = {
    "HOIST": ["ABB", "Siemens"],
    "UG_CONVEYOR": ["Sandvik", "Joy Global"],
    "MILL": ["Metso", "FLSmidth"],
    "PUMP": ["Sulzer", "Flowserve"],
    "FAN": ["Howden", "Twin City"],
    "MOTOR": ["GE", "Baldor"]
}

models = {
    "HOIST": ["HX-100", "HX-300"],
    "UG_CONVEYOR": ["CV-200", "CV-500"],
    "MILL": ["M-800", "M-1000"],
    "PUMP": ["P-40", "P-80"],
    "FAN": ["F-50", "F-75"],
    "MOTOR": ["MTR-150", "MTR-200"]
}
criticalities = ["High", "Medium", "Low"]

In [14]:
# Generate Equipment Master Data (50 rows)
equipment_data = []
for i in range(50):
    eq_class = random.choice(equipment_classes)
    manu = random.choice(manufacturers[eq_class])
    model = random.choice(models[eq_class])
    eq_id = f"EQ{i+1:03}"
    desc = f"{eq_class.replace('_', ' ').title()} Unit {i+1}"
    func_loc = f"PLANT01-{eq_class}-{i+1:03}"
    crit = random.choice(criticalities)
    equipment_data.append([eq_id, desc, func_loc, eq_class, manu, model, crit])

equipment_df = pd.DataFrame(equipment_data, columns=[
    "Equipment ID", "Description", "Functional Location", "Class",
    "Manufacturer", "Model", "Criticality"
])


In [15]:
# Display first few rows for validation
equipment_df.head()


# Generate additional datasets based on equipment_df
bom_rows = []
floc_rows = []
task_list_rows = []
fmeca_rows = []

for i, row in equipment_df.iterrows():
    # Bill of Materials - 2-4 components per equipment
    num_components = random.randint(2, 4)
    for c in range(num_components):
        bom_rows.append([
            row["Equipment ID"],
            f"COMP-{i+1:03}-{c+1}",
            f"{row['Class']} Component {c+1}",
            random.choice(["Spare", "Wear", "Critical"]),
            random.randint(1, 5)
        ])

    # Functional Location Hierarchy
    area = row["Functional Location"].split("-")[1]
    floc_rows.append([
        row["Functional Location"],
        f"{area} Area {random.randint(1, 3)}",
        f"Subarea {random.randint(1, 5)}",
        row["Class"]
    ])

    # Task List (Maintenance Plan)
    task_list_rows.append([
        row["Equipment ID"],
        f"INSPECT_{row['Class']}",
        "Inspection",
        f"Inspect {row['Class'].lower()} annually",
        "365 days"
    ])
    task_list_rows.append([
        row["Equipment ID"],
        f"LUBE_{row['Class']}",
        "Lubrication",
        f"Lubricate {row['Class'].lower()} bearings",
        "90 days"
    ])

    # FMECA data
    fmeca_rows.append([
        row["Equipment ID"],
        f"FAIL-{i+1:03}",
        f"{row['Class']} Failure Mode",
        random.choice(["Medium", "High", "Critical"]),
        random.choice(["Planned Replacement", "Condition Monitoring"]),
        row["Criticality"]
    ])

In [16]:
# Create dataframes
bom_df = pd.DataFrame(bom_rows, columns=[
    "Equipment ID", "Component ID", "Component Description", "Type", "Quantity"
])
func_loc_df = pd.DataFrame(floc_rows, columns=[
    "Functional Location", "Area", "Subarea", "Class"
])
task_list_df = pd.DataFrame(task_list_rows, columns=[
    "Equipment ID", "Task ID", "Task Type", "Description", "Frequency"
])
fmeca_df = pd.DataFrame(fmeca_rows, columns=[
    "Equipment ID", "Failure Mode ID", "Failure Mode", "Severity", "Recommended Action", "Criticality"
])


In [17]:
# Display first few rows for validation
equipment_df.head()

,Equipment ID,Description,Functional Location,Class,Manufacturer,Model,Criticality
0,EQ001,Motor Unit 1,PLANT01-MOTOR-001,MOTOR,GE,MTR-150,Low
1,EQ002,Mill Unit 2,PLANT01-MILL-002,MILL,Metso,M-800,High
2,EQ003,Motor Unit 3,PLANT01-MOTOR-003,MOTOR,GE,MTR-150,Low
3,EQ004,Pump Unit 4,PLANT01-PUMP-004,PUMP,Sulzer,P-40,High
4,EQ005,Ug Conveyor Unit 5,PLANT01-UG_CONVEYOR-005,UG_CONVEYOR,Sandvik,CV-200,Low


In [18]:
# Save to Excel in the data_templates directory
excel_path = "../data_templates/"
equipment_df.to_excel(os.path.join(excel_path, "Equipment_Master.xlsx"), index=False)
bom_df.to_excel(os.path.join(excel_path, "BOM_Template.xlsx"), index=False)
func_loc_df.to_excel(os.path.join(excel_path, "Functional_Location_Hierarchy.xlsx"), index=False)
task_list_df.to_excel(os.path.join(excel_path, "Maintenance_Task_List.xlsx"), index=False)
fmeca_df.to_excel(os.path.join(excel_path, "FMECA_Sample.xlsx"), index=False)

# Confirm files created
os.listdir(excel_path)


['Functional_Location_Hierarchy.xlsx',
 '.DS_Store',
 'FMECA_Sample.xlsx',
 'BOM_Template.xlsx',
 'Equipment_Master.xlsx',
 'Maintenance_Task_List.xlsx']

In [21]:
import plotly.graph_objects as go

# Define labels and their parent relationships to build an asset hierarchy
labels = [
    "Jansen Mine",        # Root
    "Mill Area",
    "UG Mine Area",
    "Crushing System",
    "Conveying System",
    "Crusher A",
    "Conveyor 1",
    "Motor A",
    "Gearbox A",
    "Bearing A1",
    "Seal A1"
]

parents = [
    "",                 # Jansen Mine is the root
    "Jansen Mine",      # Mill Area under Jansen Mine
    "Jansen Mine",      # UG Mine Area under Jansen Mine
    "Mill Area",        # Crushing System under Mill
    "Mill Area",        # Conveying System under Mill
    "Crushing System",  # Crusher A under Crushing
    "Conveying System", # Conveyor 1 under Conveying
    "Crusher A",        # Motor A under Crusher A
    "Motor A",          # Gearbox under Motor A
    "Gearbox A",        # Bearing under Gearbox
    "Gearbox A"         # Seal under Gearbox
]

# Create the treemap figure
fig = go.Figure(go.Treemap(
    labels=labels,
    parents=parents,
    root_color="lightgrey"
))

# Update layout for spacing and title
fig.update_layout(
    title="Asset Hierarchy – Jansen Mine Example",
    margin=dict(t=30, l=10, r=10, b=10)
)

# Show the figure
fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Treemap: 'root_color'

    Valid properties:
        branchvalues
            Determines how the items in `values` are summed. When
            set to "total", items in `values` are taken to be value
            of all its descendants. When set to "remainder", items
            in `values` corresponding to the root and the branches
            sectors are taken to be the extra part not part of the
            sum of the values at their leaves.
        count
            Determines default for `values` when it is not
            provided, by inferring a 1 for each of the "leaves"
            and/or "branches", otherwise 0.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets 